In [46]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [47]:
# Load real dataset instead of generating synthetic data
df = pd.read_csv("expanded_simulated_fitness_data.csv")

# Extract relevant columns
heart_rate = df['heart_rate_bpm']
spo2 = df['spo2_percent']
time_spent_skating = df['time_duration_min']

In [48]:

# Load real dataset instead of generating synthetic data
import pandas as pd
df = pd.read_csv("expanded_simulated_fitness_data.csv")

# Extract relevant columns
heart_rate = df['heart_rate_bpm']
spo2 = df['spo2_percent']
time_spent_skating = df['time_duration_min']

In [49]:
# Define performance levels based on HR, SpO2, and time spent skating
def classify_performance(hr, sp, time):
    if hr < 100 and sp > 96 and time > 30:
        return "Elite"
    elif 100 <= hr < 130 and sp > 94 and time > 20:
        return "Advanced"
    elif 130 <= hr < 160 and sp > 92 and time > 10:
        return "Intermediate"
    else:
        return "Beginner"

performance_levels = [classify_performance(hr, sp, time) for hr, sp, time in zip(heart_rate, spo2, time_spent_skating)]

In [50]:

# Define risk levels based on HR, SpO2, and time spent skating
def classify_risk(hr, sp, time):
    if hr >= 150 or sp < 88 or time > 100:
        return "High Risk"
    elif (120 <= hr < 150) or (88 <= sp < 92) or (80 <= time <= 100):
        return "Mid Risk"
    else:
        return "Low Risk"

risk_levels = [classify_risk(hr, sp, time) for hr, sp, time in zip(heart_rate, spo2, time_spent_skating)]

In [51]:

# Create DataFrame
df = pd.DataFrame({
    "Heart_Rate": heart_rate,
    "SpO2": spo2,
    "Time_Spent_Skating": time_spent_skating,
    "Performance_Level": performance_levels,
    "Risk_Level": risk_levels
})

In [52]:
# Encode target variables (performance levels and risk levels)
df["Performance_Level"] = df["Performance_Level"].astype("category").cat.codes
df["Risk_Level"] = df["Risk_Level"].astype("category").cat.codes

In [53]:
# Add 10% noise to the performance labels
def add_label_noise(labels, noise_level=0.1):
    labels = labels.copy()
    unique_labels = labels.unique()
    n = int(len(labels) * noise_level)
    noisy_indices = np.random.choice(labels.index, size=n, replace=False)
    for idx in noisy_indices:
        original = labels[idx]
        others = [lbl for lbl in unique_labels if lbl != original]
        labels.at[idx] = np.random.choice(others)
    return labels

df['noisy_performance_level'] = add_label_noise(df['Performance_Level'])
df['noisy_risk_level'] = add_label_noise(df['Risk_Level'])

In [54]:
# Split the dataset again with noisy labels
y_noisy_performance = df["noisy_performance_level"]
y_noisy_risk = df["noisy_risk_level"]

X_train, X_test, y_train_noisy_perf, y_test_noisy_perf = train_test_split(X, y_noisy_performance, test_size=0.2, random_state=42)
X_train, X_test, y_train_noisy_risk, y_test_noisy_risk = train_test_split(X, y_noisy_risk, test_size=0.2, random_state=42)


In [55]:

# Retrain classifiers with noisy labels
clf_noisy_performance = RandomForestClassifier(n_estimators=100, random_state=42)
clf_noisy_risk = RandomForestClassifier(n_estimators=100, random_state=42)

clf_noisy_performance.fit(X_train, y_train_noisy_perf)
clf_noisy_risk.fit(X_train, y_train_noisy_risk)

RandomForestClassifier(random_state=42)

In [56]:
# Predict and evaluate performance classification with noise
y_pred_noisy_perf = clf_noisy_performance.predict(X_test)
accuracy_noisy_perf = accuracy_score(y_test_noisy_perf, y_pred_noisy_perf)

In [57]:
# Predict and evaluate risk classification with noise
y_pred_noisy_risk = clf_noisy_risk.predict(X_test)
accuracy_noisy_risk = accuracy_score(y_test_noisy_risk, y_pred_noisy_risk)

In [58]:
# Display results with noise
print(f"Performance Classification Accuracy with Noise: {accuracy_noisy_perf}")
print(f"Risk Classification Accuracy with Noise: {accuracy_noisy_risk}")

# Show sample data
print("\nSample Data with Risk Levels:")
print(df.drop(columns=["noisy_performance_level","noisy_risk_level"]).head(10))

Performance Classification Accuracy with Noise: 0.8727272727272727
Risk Classification Accuracy with Noise: 0.8931818181818182

Sample Data with Risk Levels:
   Heart_Rate  SpO2  Time_Spent_Skating  Performance_Level  Risk_Level
0         131  96.0                  43                  3           2
1         176  96.0                  28                  1           0
2         123  99.9                  37                  0           2
3         101  90.5                  20                  1           2
4         110  93.8                  56                  1           1
5         132  90.7                   8                  1           2
6         112  92.3                  32                  1           1
7          67  90.3                  18                  1           2
8          61  94.3                  58                  1           1
9          77  97.8                  38                  2           1
